In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install Sastrawi
%pip install google-play-scraper

In [ ]:
from google_play_scraper import app
import pandas as pd
import numpy as np

In [ ]:
from google_play_scraper import Sort, reviews

result, continuation_token = reviews(
    'com.fws.plantsnap2', # ID = google play
    lang='id',
    country='id',
    sort=Sort.MOST_RELEVANT,
    count=200,
    filter_score_with= 5 # defaults to None(means all score) Use 1 or 2 or 3 or 4 or 5 to select certain score
)

In [ ]:
# Dataframe dengan nama
df_ulasan = pd.DataFrame(np.array(result),columns=['review'])
df_ulasan = df_ulasan.join(pd.DataFrame(df_ulasan.pop('review').tolist()))
df_ulasan.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,5060d678-92ec-43ee-be36-b943dd0247ec,Syamsul Arifin,https://play-lh.googleusercontent.com/a-/ALV-U...,Sangat terbantu dengan adanya aplikasi ini,5,0,6.2.4,2023-12-11 02:27:05,None,NaT,6.2.4
1,5c1519a9-9e31-418b-a9f6-0ec69954909f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Saya penjual tanaman hias yang baru memulai us...,5,8,None,2019-10-11 05:51:13,"Hello Trisna, Thank you for such valuable feed...",2019-10-11 09:56:45,None
2,b5aa3dac-e3ed-48b4-8c09-bd665b574a43,Abeng Fariz,https://play-lh.googleusercontent.com/a-/ALV-U...,"Amazing sih ini, keren gila. Saya respectful p...",5,16,3.00.26,2020-05-19 18:40:06,None,NaT,3.00.26
3,2db7641a-036d-4ab3-bb7c-5f7c28ac9d87,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kalo ada yang bilang ini aplikasi yang tidak b...,5,4,None,2018-12-16 08:29:17,Thanks for being a part of PlantSnap! Please w...,2018-12-18 00:42:25,None
4,2e4eb94f-74a5-4688-9c68-c451ac6f5711,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,aplikasi canggih yang sangatlah membantu buat ...,5,6,3.00.20,2020-03-12 10:51:08,"Hello Emak, Thanks for being a part of PlantSn...",2020-03-12 12:16:11,3.00.20


In [ ]:


df_ulasan[['content', 'score']].head()



,content,score
0,Sangat terbantu dengan adanya aplikasi ini,5
1,Saya penjual tanaman hias yang baru memulai us...,5
2,"Amazing sih ini, keren gila. Saya respectful p...",5
3,kalo ada yang bilang ini aplikasi yang tidak b...,5
4,aplikasi canggih yang sangatlah membantu buat ...,5


In [ ]:
my_ulasan = df_ulasan[['content', 'score']]

In [ ]:


my_ulasan.to_csv("riliv-rating5.csv", index = False)



In [ ]:
%ls

drive/  ratingAll.csv  riliv-rating1.csv  riliv-rating3.csv  riliv-rating5.csv  sample_data/


In [ ]:
df_rating5 = pd.read_csv("riliv-rating5.csv")
df_rating3 = pd.read_csv("riliv-rating3.csv")
df_rating1 = pd.read_csv("riliv-rating1.csv")

In [ ]:
df_ratingAll = pd.concat([df_rating5,df_rating3,df_rating1])

In [ ]:
len(df_ratingAll)

420

In [ ]:


df_ratingAll.to_csv("ratingAll.csv", index = False)



In [ ]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


# Load the dataset
dframe = pd.read_csv("ratingAll.csv")

In [ ]:
# Case Folding
data_casefolded = dframe['content'].str.lower()

In [ ]:
# Filtering
def filter_text(text):
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)  # Remove URLs
    text = re.sub(r'\(cont\)', ' ', text)  # Remove continuations
    text = re.sub(r'[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', text)  # Remove punctuation
    text = re.sub(r'#\S+', '', text)  # Remove hashtags
    text = re.sub(r'\d+', ' ', text)  # Remove numbers
    return text

data_filtered = data_casefolded.apply(filter_text)

In [ ]:
# Tokenization
nltk.download('punkt')
data_tokenized = data_filtered.apply(word_tokenize)

print(data_tokenized)

0      [sangat, terbantu, dengan, adanya, aplikasi, ini]
1      [saya, penjual, tanaman, hias, yang, baru, mem...
2      [amazing, sih, ini, keren, gila, saya, respect...
3      [kalo, ada, yang, bilang, ini, aplikasi, yang,...
4      [aplikasi, canggih, yang, sangatlah, membantu,...
                             ...                        
415                               [out, of, expectation]
416                                      [sering, salah]
417                                           [ga, tahu]
418                                       [tidak, keren]
419                                             [mantep]
Name: content, Length: 420, dtype: object


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Stopword Removal
factory = StopWordRemoverFactory()
stopword_list = factory.get_stop_words()

def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopword_list and len(word) > 3]

data_no_stopwords = data_tokenized.apply(remove_stopwords)

print(data_no_stopwords)

0                   [sangat, terbantu, adanya, aplikasi]
1      [penjual, tanaman, hias, baru, memulai, usaha,...
2      [amazing, keren, gila, respectful, developer, ...
3      [kalo, bilang, aplikasi, berguna, mohon, pikir...
4      [aplikasi, canggih, sangatlah, membantu, buat,...
                             ...                        
415                                        [expectation]
416                                      [sering, salah]
417                                               [tahu]
418                                              [keren]
419                                             [mantep]
Name: content, Length: 420, dtype: object


In [ ]:
path_dataslang = open("/content/drive/MyDrive/Analisis Sentimen/kamus.csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

ulasan_formal = []
for data in data_no_stopwords:
  data_clean = [replaceSlang(word) for word in data]
  ulasan_formal.append(data_clean)
len_ulasan_formal = len(ulasan_formal)
print(ulasan_formal)
len_ulasan_formal

[['sangat', 'terbantu', 'adanya', 'aplikasi'], ['penjual', 'tanaman', 'hias', 'baru', 'memulai', 'usaha', 'awalnya', 'sangat', 'bingung', 'keringat', 'persatu', 'nama', 'tanaman', 'berbelanja', 'tanaman', 'menjualnya', 'mengenal', 'nama', 'lalu', 'coba', 'mencari', 'cari', 'aplikasinya', 'alhamdulillah', 'menemukan', 'disini', 'sekarang', 'perlu', 'bingung', 'mengaktifkan', 'kamera', 'mengarahkan', 'tanaman', 'tahu', 'terimakasih', 'plant', 'snap'], ['amazing', 'keren', 'gila', 'respectful', 'developer', 'terus', 'dikembangkan', 'jangan', 'patah', 'semangat', 'menyelaraskan', 'alam', 'teknologi', 'beli', 'premium', 'salam', 'lestari'], ['kalau', 'bilang', 'aplikasi', 'berguna', 'mohon', 'pikir', 'jangan', 'mengandalkan', 'aplikasi', 'global', 'menganalisa', 'disekitar', 'tinggal', 'diindonesia', 'menggunakan', 'aplikasi', 'berjalan', 'sesuai', 'harapan', 'sepenuhnya', 'bukan', 'kesalahan', 'aplikasi', 'ingat', 'keberagaman', 'bahasa', 'indonesia', 'membuat', 'analisa', 'nama', 'jenis',

420

In [ ]:
factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
white_list = ["bali"]

def stemmer(line):
  valid = list()
  for word in line:
    if(word not in white_list):
      word = ind_stemmer.stem(word)
    if(len(word)>2):
      valid.append(word)
  return valid

data_stemmed = [stemmer (line) for line in ulasan_formal]
print(data_stemmed)

[['sangat', 'bantu', 'ada', 'aplikasi'], ['jual', 'tanam', 'hias', 'baru', 'mulai', 'usaha', 'awal', 'sangat', 'bingung', 'keringat', 'satu', 'nama', 'tanam', 'belanja', 'tanam', 'jual', 'kenal', 'nama', 'lalu', 'coba', 'cari', 'cari', 'aplikasi', 'alhamdulillah', 'temu', 'sini', 'sekarang', 'perlu', 'bingung', 'aktif', 'kamera', 'arah', 'tanam', 'tahu', 'terimakasih', 'plant', 'snap'], ['amazing', 'keren', 'gila', 'respectful', 'developer', 'terus', 'kembang', 'jangan', 'patah', 'semangat', 'selaras', 'alam', 'teknologi', 'beli', 'premium', 'salam', 'lestari'], ['kalau', 'bilang', 'aplikasi', 'guna', 'mohon', 'pikir', 'jangan', 'andal', 'aplikasi', 'global', 'menganalisa', 'sekitar', 'tinggal', 'indonesia', 'guna', 'aplikasi', 'jalan', 'sesuai', 'harap', 'sepenuh', 'bukan', 'salah', 'aplikasi', 'ingat', 'agam', 'bahasa', 'indonesia', 'buat', 'analisa', 'nama', 'jenis', 'tumbuh', 'aplikasi', 'sepenuh', 'sama', 'nama', 'jenis', 'tumbuh', 'daerah'], ['aplikasi', 'canggih', 'sangat', 'ban

In [ ]:
unique_words = []
for word in ulasan_formal:
    if word not in unique_words:
        unique_words.append(word)

# Menggabungkan kembali kata-kata unik menjadi teks
unique_text = unique_words

print(unique_text)

[['sangat', 'terbantu', 'adanya', 'aplikasi'], ['penjual', 'tanaman', 'hias', 'baru', 'memulai', 'usaha', 'awalnya', 'sangat', 'bingung', 'keringat', 'persatu', 'nama', 'tanaman', 'berbelanja', 'tanaman', 'menjualnya', 'mengenal', 'nama', 'lalu', 'coba', 'mencari', 'cari', 'aplikasinya', 'alhamdulillah', 'menemukan', 'disini', 'sekarang', 'perlu', 'bingung', 'mengaktifkan', 'kamera', 'mengarahkan', 'tanaman', 'tahu', 'terimakasih', 'plant', 'snap'], ['amazing', 'keren', 'gila', 'respectful', 'developer', 'terus', 'dikembangkan', 'jangan', 'patah', 'semangat', 'menyelaraskan', 'alam', 'teknologi', 'beli', 'premium', 'salam', 'lestari'], ['kalau', 'bilang', 'aplikasi', 'berguna', 'mohon', 'pikir', 'jangan', 'mengandalkan', 'aplikasi', 'global', 'menganalisa', 'disekitar', 'tinggal', 'diindonesia', 'menggunakan', 'aplikasi', 'berjalan', 'sesuai', 'harapan', 'sepenuhnya', 'bukan', 'kesalahan', 'aplikasi', 'ingat', 'keberagaman', 'bahasa', 'indonesia', 'membuat', 'analisa', 'nama', 'jenis',

In [ ]:
# Join tokens back to string
data_preprocessed = [' '.join(tokens) for tokens in data_stemmed]
print(data_preprocessed)
#data_steammed berfungsi untuk mengubah data yang awalan array perkata menjadi satu kalimat

['sangat bantu ada aplikasi', 'jual tanam hias baru mulai usaha awal sangat bingung keringat satu nama tanam belanja tanam jual kenal nama lalu coba cari cari aplikasi alhamdulillah temu sini sekarang perlu bingung aktif kamera arah tanam tahu terimakasih plant snap', 'amazing keren gila respectful developer terus kembang jangan patah semangat selaras alam teknologi beli premium salam lestari', 'kalau bilang aplikasi guna mohon pikir jangan andal aplikasi global menganalisa sekitar tinggal indonesia guna aplikasi jalan sesuai harap sepenuh bukan salah aplikasi ingat agam bahasa indonesia buat analisa nama jenis tumbuh aplikasi sepenuh sama nama jenis tumbuh daerah', 'aplikasi canggih sangat bantu buat khusus anak anak lebih cepat tahu jenis tanam pasti beri tambah ilmu sangat guna tahu', 'aplikasi sensor detektor tanam sangat bantu mudah segala khusus kenal ajar jelajah banyak kehiduoan sekitar kita', 'aplikasi bagus banget sangat bantu mahasiswa biologi', 'suka banget aplikasi temu pr

In [ ]:
# Feature Extraction with TFIdf
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data_preprocessed)
y = dframe['score']

In [ ]:
import pickle
pickle.dump(vectorizer, open("cobaa.pickle", "wb"))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)  # tentukan jumlah tetangga (K)
knn.fit(X_train, y_train)

# Prediksi data uji
knn_y_pred = knn.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, knn_y_pred)
print(f'Akurasi model KNN: {accuracy:.2f}')

report_knn = classification_report(y_test, knn_y_pred)
print ('report : ' +str(report_knn))

Akurasi model KNN: 0.60
report :               precision    recall  f1-score   support

           1       0.50      0.04      0.08        23
           3       0.86      0.35      0.50        17
           5       0.57      0.98      0.72        44

    accuracy                           0.60        84
   macro avg       0.64      0.46      0.43        84
weighted avg       0.61      0.60      0.50        84



In [ ]:
from sklearn import svm

# Inisialisasi dan latih model SVM
svm = svm.SVC(kernel='linear')
svm.fit(X_train, y_train)

# Prediksi data uji
y_pred = svm.predict(X_test)

# Evaluasi model
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi model SVM: {accuracy:.2f}')

report = classification_report(y_test, y_pred)
print ('report : ' +str(report))

Akurasi model SVM: 0.70
report :               precision    recall  f1-score   support

           1       0.75      0.52      0.62        23
           3       0.78      0.41      0.54        17
           5       0.68      0.91      0.78        44

    accuracy                           0.70        84
   macro avg       0.74      0.61      0.64        84
weighted avg       0.72      0.70      0.68        84



In [ ]:
import pickle

# Save the model
with open('reviews.pickle', 'wb') as file:
    pickle.dump(data_preprocessed, file)

print("Model saved successfully.")

Model saved successfully.


In [ ]:


import pickle

# Save the model
with open('svm_model.pkl', 'wb') as file:
    pickle.dump(svm, file)

print("Model saved successfully.")



Model saved successfully.
